### Извлечение сущностей с использованием Natasha и Yargy-parser

#### Готовые правила
Сейчас в Natasha есть правила для извлечения имён, адресов, дат, сумм денег, названий организаций и географических объектов.

#### Имена

In [ ]:
!pip install natasha==0.10.0 yargy==0.11.0

In [ ]:
from natasha import NamesExtractor
from natasha.markup import show_markup, show_json

extractor = NamesExtractor()

text = '''
Благодарственное письмо   
Хочу поблагодарить учителей моего, теперь уже бывшего, одиннадцатиклассника:  Бушуева Вячеслава Владимировича и Бушуеву Веру Константиновну. 
Они вовлекали сына в интересные внеурочные занятия, связанные с театром и походами.

Благодарю прекрасного учителя 1"А" класса - Волкову Наталью Николаевну, нашего наставника, тьютора - Ларису Ивановну!
За огромнейший труд, чуткое отношение к детям, взаимопонимание! Огромное спасибо!
'''

matches = extractor(text)
spans = [_.span for _ in matches]
print('Spans: ',spans)
facts = [_.fact.as_json for _ in matches]
print('Facts: ',facts)
show_markup(text, spans)
show_json(facts)


#### Адреса

In [ ]:
from natasha import AddressExtractor
from natasha.markup import show_markup, show_json

extractor = AddressExtractor()

text = '''
Предлагаю вернуть прежние границы природного парка №71 на Инженерной улице 2.

По адресу Алтуфьевское шоссе д.51 (основной вид разрешенного использования: производственная деятельность, склады) размещен МПЗ. Жители требуют незамедлительной остановки МПЗ и его вывода из района

Контакты О нас телефон 7 881 574-10-02 Адрес Республика Карелия,г.Петрозаводск,ул.Маршала Мерецкова, д.8 Б,офис 4
'''
matches = extractor(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
show_json(facts)

#### Правила для дат и денег

In [ ]:
from natasha import (
    NamesExtractor,
    AddressExtractor,
    DatesExtractor,
    MoneyExtractor
)
from natasha.markup import show_markup, show_json

extractors = [
    NamesExtractor(),
    AddressExtractor(),
    DatesExtractor(),
    MoneyExtractor()
]


text = '''
Взыскать к индивидуального предпринимателя Иванова Костантипа Петровича дата рождения 10 января 1970 года, проживающего по адресу город Санкт-Петербург, ул. Крузенштерна, дом 5/1А 8 000 (восемь тысяч) рублей 00 копеей госпошлины в пользу бюджета РФ 
'''
spans = []
facts = []
for extractor in extractors:
    matches = extractor(text)
    spans.extend(_.span for _ in matches)
    facts.extend(_.fact.as_json for _ in matches)
show_markup(text, spans)
show_json(facts)


#### Создание своих правил

#### Правило для извлечения дат в ISO-формате («2018-02-23», «2015-12-31»)

In [1]:
from yargy import rule, and_, Parser
from yargy.predicates import gte, lte


DAY = and_(
    gte(1),
    lte(31)
)
MONTH = and_(
    gte(1),
    lte(12)
)
YEAR = and_(
    gte(1),
    lte(2022)
)
DATE = rule(
    YEAR,
    '-',
    MONTH,
    '-',
    DAY
)

parser = Parser(DATE)
text = '''
2018-02-23,
2015-12-31;
8 916 364-12-01'''
for match in parser.findall(text):
    print(match.span, [_.value for _ in match.tokens])

[1, 11) ['2018', '-', '02', '-', '23']
[13, 23) ['2015', '-', '12', '-', '31']
[31, 40) ['364', '-', '12', '-', '01']


#### Корректировка правила

In [6]:
from yargy import rule, and_, Parser
from yargy.predicates import gte, lte


DAY = and_(
    gte(1),
    lte(31)
)
MONTH = and_(
    gte(1),
    lte(12)
)
YEAR = and_(
    gte(2000),
    lte(2022)
)
DATE = rule(
    YEAR,
    '-',
    MONTH,
    '-',
    DAY
)

parser = Parser(DATE)
text = '''
2018-02-23,
2015-12-31;
8 916 364-12-01'''
for match in parser.findall(text):
    print(match.span, [_.value for _ in match.tokens])

[1, 11) ['2018', '-', '02', '-', '23']
[13, 23) ['2015', '-', '12', '-', '31']


#### Правила для дат вида «8 января 2014», «15 июня 2001 г.»

In [11]:
from yargy import or_
from yargy.predicates import caseless, normalized, dictionary


MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'мая',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
}
MONTH_NAME = dictionary(MONTHS)
YEAR_WORDS = or_(
    rule(caseless('г'), '.'),
    rule(normalized('год'))
)
DATE = or_(
    rule(
        YEAR,
        '-',
        MONTH,
        '-',
        DAY
    ),
    rule(
        DAY,
        MONTH_NAME,
        YEAR,
        YEAR_WORDS.optional()
    )
)

parser = Parser(DATE)
text = '''
8 января 2014 года, 15 июня 2001 г.,
31 февраля 2018'''
for match in parser.findall(text):
    print(match.span, [_.value for _ in match.tokens])

[1, 19) ['8', 'января', '2014', 'года']
[21, 36) ['15', 'июня', '2001', 'г', '.']
[38, 53) ['31', 'февраля', '2018']


#### Интерпретация

In [ ]:
from yargy.interpretation import fact


Date = fact(
    'Date',
    ['year', 'month', 'day', 'year_words']
)

DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day
)
MONTH = and_(
    gte(1),
    lte(12)
).interpretation(
    Date.month
)
YEAR = and_(
    gte(1),
    lte(2018)
).interpretation(
    Date.year
)
MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Date.month
)
YEAR_WORDS = or_(
    rule(caseless('г'), '.'),
    rule(normalized('год'))
).interpretation(
    Date.year_words
)
DATE = or_(
    rule(YEAR, '-', MONTH, '-', DAY),
    rule(
        DAY,
        MONTH_NAME,
        YEAR,
        YEAR_WORDS
    )
).interpretation(Date)

parser = Parser(DATE)
text = '''5 февраля 2011 года'''
for match in parser.findall(text):
    print(match.fact)


In [ ]:
match = parser.match('2018-12-01')
match.tree.as_dot

### Извлечение данных из резюме сосискателей

#### Использование Yargy-парсера для извлечения пола, возраста, даты рождения, места проживания, гражданства и разрешения на работу

In [ ]:
#клонируем репозиторий гитхаб
!git clone -l -s https://github.com/Ekaterina-Elmikeeva/Text-analytics.git
%cd Text-analytics/resources/resume_texts
!ls

### Данные

In [ ]:
#читаем файлы с резюме соискателей
import os
ls = [i for i in os.listdir() if i.endswith('.txt')]
ls.sort()

texts = []
for path in ls:
    with open(path) as file:
        text = file.read()
        texts.append(text)
        
intros = [_[:700] for _ in sorted(texts)]

intros

In [ ]:
# 3 примера резюме
from random import seed, sample

seed(41)
for text in sample(intros, 3):
    print(text)
    print('---' * 10)

### Установка библиотек и определение методов

In [ ]:
!pip install ipymarkup

In [ ]:
# импортируем необходимые билиотеки
import json

from ipymarkup import show_span_ascii_markup as show_markup

from yargy import (
    Parser,
    rule, or_, and_, not_
)
from yargy.predicates import (
    eq, in_,
    type, normalized,
    dictionary,
    gte, lte
)
from yargy.pipelines import (
    pipeline,
    morph_pipeline
)
from yargy.interpretation import (
    fact,
    attribute
)
from yargy.tokenizer import MorphTokenizer, EOL


# определяем структуру данных, которая нам нужна
Intro = fact(
    'Intro',
    ['gender', 'age', 'birth', 'location'
    ,
     'citizenship', 'permission'
    ]
)

INT = type('INT')
COMMA = eq(',')
COLON = eq(':')


def show_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))


def show_matches(rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        spans = [_.span for _ in matches]
        show_markup(line, spans)

### Пол

In [ ]:
GENDERS = {
    'Женщина': 'female',
    'Мужчина': 'male'
}

GENDER = in_(GENDERS).interpretation(
    Intro.gender.custom(GENDERS.get)
)


show_matches(
    GENDER,
    'мужчина, Мужчина, мужчину',
    'Женщина'
)

### Возраст

In [ ]:
AGE = rule(
    INT,
    normalized('год')
)


show_matches(
    AGE,
    '21 год, 25 лет'
)

### Дата рождения

In [ ]:
Date = fact(
    'Date',
    ['year', 'month', 'day']
)


AGE = rule(
    INT.interpretation(
        Intro.age.custom(int)
    ),
    normalized('год')
)

MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'май': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12
}

MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Date.month.normalized().custom(MONTHS.get)
)

DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day.custom(int)
)

YEAR = and_(
    gte(1900),
    lte(2100)
).interpretation(
    Date.year.custom(int)
)

DATE = rule(
    DAY,
    MONTH_NAME,
    YEAR
).interpretation(
    Date
)

BIRTH = rule(
    normalized('родиться'),
    DATE.interpretation(
        Intro.birth
    )
)


show_matches(
    BIRTH,
    'родился 21 февраля 1990',
    'родиться 32 сентябрь 2000',
    'родилась 01 июля 1917',
)

### Соц-дем правило

In [ ]:
SOCDEM = rule(
    GENDER, COMMA,
    AGE, COMMA,
    BIRTH
)


parser = Parser(SOCDEM)
seed(10)
for text in sample(intros, 10):
    for match in parser.findall(text):
        start, stop = match.span
        print(text[start:stop])

### Место проживания

In [ ]:
def load_lines(path):
    with open(path) as file:
        for line in file:
            yield line.rstrip('\n')

In [ ]:
#исползуем 2 готовых справочника - со станциями метро и районами
METRO_STATIONS = set(load_lines('/content/Text-analytics/resources/resume_dicts/metro.txt'))
AREAS = set(load_lines('/content/Text-analytics/resources/resume_dicts/areas.txt'))
seed(10)
sample(sorted(METRO_STATIONS), 10), sample(sorted(AREAS), 10)

In [ ]:
Location = fact(
    'Location',
    ['area', 'metro']
)


METRO = rule(
    'м', '.',
    pipeline(METRO_STATIONS).interpretation(
        Location.metro
    )
)

AREA = pipeline(AREAS).interpretation(
    Location.area
)

LOCATION = rule(
    AREA,
    rule(
        COMMA,
        METRO
    ).optional()
).interpretation(
    Location
)


show_matches(
    LOCATION,
    'место проживания: Москва, м. Парк Победы',
    'Киев, м.Киевская',
    'Россия',
    'в Москве',
    'м. парк победы',
    'на м. Кропоткинской',
)

In [ ]:
TITLE = rule(
    normalized('проживает'), COLON
)

LIVES_AT = rule(
    TITLE,
    LOCATION
)


parser = Parser(LIVES_AT)
seed(10)
for text in sample(intros, 10):
    for match in parser.findall(text):
        start, stop = match.span
        print(text[start:stop])

### Гражданство

In [ ]:
TITLE = rule(
    'Гражданство', COLON
)

ITEM = AREA.interpretation(
    Intro.citizenship
)

LOCATIONS = rule(
    ITEM,
    rule(
        COMMA,
        ITEM
    ).optional()
)

CITIZENSHIP = rule(
    TITLE,
    LOCATIONS
)


show_matches(
    CITIZENSHIP,
    'Гражданство: Россия, Франция',
    'Гражданство: Россия, Франция, Украина',
)

### Наличие разрешения на работу

In [ ]:
TITLE = pipeline([
    'есть разрешение на работу:'
])

ITEM = AREA.interpretation(
    Intro.permission
)

LOCATIONS = rule(
    ITEM,
    rule(
        COMMA,
        ITEM
    ).optional().repeatable()
)

PERMISSION = rule(
    TITLE,
    LOCATIONS
)


show_matches(
    PERMISSION,
    'есть разрешение на работу: Россия, Франция, Украина',
    'есть разрешение на работу: Россия',
)


### Результаты

In [ ]:
INTRO = rule(
    SOCDEM,
    LIVES_AT,
    CITIZENSHIP, COMMA, PERMISSION
).interpretation(
    Intro
)


parser = Parser(INTRO, tokenizer=TOKENIZER)
seed(10)
for text in sample(intros, 10):
    for match in parser.findall(text):
        start, stop = match.span
        print(text[start:stop])
        print()